In [7]:
# basic maths capabilities
import math

# more advanced maths capabilities
import numpy
import numpy.random as rnd
import networkx
# timing how long things take
import time

# analysis and presentation
import pandas
# graphing and animation tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
from matplotlib import animation
import seaborn as sns

# use high-performance SVG graphics in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [8]:
class GraphWithDynamics(networkx.Graph):
    '''A NetworkX undirected network with an associated dynamics. This
    class combines two sets of entwined functionality: a network, and
    the dynamical process being studied. This is the abstract base class
    for studying different kinds of dynamics.'''

    # keys for node and edge attributes
    OCCUPIED = 'occupied'     # edge has been used to transfer infection or not
    DYNAMICAL_STATE = 'sir'   # dynamical state of a node

    def __init__( self, g = None ):
        '''Create a graph, optionally with nodes and edges copied from
        the graph given.
        
        g: graph to copy (optional)'''
        networkx.Graph.__init__(self, g)
        if g is not None:
            self.copy_from(g)
        
    def copy_from( self, g ):
        '''Copy the nodes and edges from another graph into us.
        
        g: the graph to copy from
        returns: the graph'''
        
        # copy in nodes and edges from source network
        self.add_nodes_from(g.nodes_iter())
        self.add_edges_from(g.edges_iter())
        
        # remove self-loops
        es = self.selfloop_edges()
        self.remove_edges_from(es)
        
        return self
    
    def remove_all_nodes( self ):
        '''Remove all nodes and edges from the graph.'''
        self.remove_nodes_from(self.nodes())

    def at_equilibrium( self, t ):
        '''Test whether the model is an equilibrium. The default runs for
        20000 steps and then stops.
        
        t: the current simulation timestep
        returns: True if we're done'''
        return (t >= 20000)

    def before( self ):
        '''Placeholder to be run ahead of simulation, Defaults does nothing.'''
        pass

    def after( self ):
        '''Placeholder to be run after simulation, Defaults does nothing.'''
        pass
    
    def _dynamics( self ):
        '''Internal function defining the way the dynamics works.

        returns: a dict of properties'''
        raise NotYetImplementedError('_dynamics()')
        
    def dynamics( self ):
        '''Run a number of iterations of the model over the network. The
        default doesn't add anything to the basic parameters captured 
        
        returns: a dict of properties'''
        return self._dynamics()

    def skeletonise( self ):
        '''Remove unoccupied edges from the network.
        
        returns: the network with unoccupied edges removed'''
        
        # find all unoccupied edges
        edges = []
        for n in self.nodes_iter():
            for (np, m, data) in self.edges_iter(n, data = True):
                if (self.OCCUPIED not in data.keys()) or (data[self.OCCUPIED] != True):
                    # edge is unoccupied, mark it to be removed
                    # (safe because there are no parallel edges)
                    edges.insert(0, (n, m))
                    
        # remove them
        self.remove_edges_from(edges)
        return self
    
    def populations( self ):
        '''Return a count of the number of nodes in each dynamical state.
        
        returns: a dict'''
        pops = dict()
        for n in self.nodes_iter():
            s = self.node[n][self.DYNAMICAL_STATE]
            if s not in pops.keys():
                pops[s] = 1
            else:
                pops[s] = pops[s] + 1
        return pops

In [9]:
class GraphWithSynchronousDynamics(GraphWithDynamics):
    '''A graph with a dynamics that runs synchronously,
    applying the dynamics to each node in the network.'''
        
    def __init__( self, g = None ):
        '''Create a graph, optionally with nodes and edges copied from
        the graph given.
        
        g: graph to copy (optional)'''
        GraphWithDynamics.__init__(self, g)
        
    def model( self, n ):
        '''The dynamics function that's run over the network. This
        is a placeholder to be re-defined by sub-classes.
        
        n: the node being simulated
        returns: the number of events that happened in this timestep'''
        raise NotYetImplementedError('model()')
    
    def _dynamics_step( self, t ):
        '''Run a single step of the model over the network.
        
        t: timestep being simulated
        returns: the number of dynamic events that happened in this timestep'''
        events = 0
        for i in self.node.keys():
            events = events + self.model(i)
        return events

    def _dynamics( self ):
        '''Synchronous dynamics. We apply _dynamics_step() at each timestep
        and then check for completion using at_equilibrium().
        
        returns: a dict of simulation properties'''
        rc = dict()

        rc['start_time'] = time.clock()
        self.before()
        t = 0
        events = 0
        eventDist = dict()
        timestepEvents = 0
        while True:
            # run a step
            nev = self._dynamics_step(t)
            if nev > 0:
                events = events + nev
                timestepEvents = timestepEvents + 1
                eventDist[t] = nev
        
            # test for termination
            if self.at_equilibrium(t):
                break
            
            t = t + 1
        self.after()
        rc['end_time'] = time.clock()
        
        # return the simulation-level results
        rc['elapsed_time'] = rc['end_time'] - rc['start_time']
        rc['timesteps'] = t
        rc['events'] = events
        rc['event_distribution'] = eventDist
        rc['timesteps_with_events'] = timestepEvents
        return rc

In [10]:
class SIRSynchronousDynamics(GraphWithSynchronousDynamics):
    '''A graph with a particular SIR dynamics. We use probabilities
    to express infection and recovery per timestep, and run the system
    using synchronous dynamics.'''
    
    # the possible dynamics states of a node for SIR dynamics
    SUSCEPTIBLE = 'S'
    INFECTED = 'I'
    RECOVERED = 'R'
    
    # list of infected nodes, the sites of all the dynamics
    _infected = []
    
    # list of dynamic states captured during a simulation
    _states = dict()
    
    def __init__( self, pInfect = 0.0, pRecover = 1.0, pInfected = 0.0, g = None ):
        '''Generate a graph with dynamics for the given parameters.
        
        pInfect: infection probability (defaults to 0.0)
        pRecover: probability of recovery (defaults to 1.0)
        pInfected: initial infection probability (defaults to 0.0)
        g: the graph to copy from (optional)'''
        GraphWithSynchronousDynamics.__init__(self, g)
        self._pInfect = pInfect
        self._pRecover = pRecover
        self._pInfected = pInfected
            
    def before( self ):
        '''Seed the network with infected nodes, and mark all edges
        as unoccupied by the dynamics.'''
        self._infected = []       # in case we re-run from a dirty intermediate state
        for n in self.node.keys():
            if numpy.random.random() <= self._pInfected:
                self._infected.insert(0, n)
                self.node[n][self.DYNAMICAL_STATE] = self.INFECTED
            else:
                self.node[n][self.DYNAMICAL_STATE] = self.SUSCEPTIBLE
        for (n, m, data) in self.edges_iter(data = True):
            data[self.OCCUPIED] = False

    def _dynamics_step( self, t ):
        '''Optimised per-step dynamics that only runs the dynamics at infected
        nodes, since they're the only places where state changes originate. At the
        end of each timestep we re-build the infected node list.
        
        t: timestep being simulated
        returns: the number of events that happened in this timestep'''
        events = 0
        
        # run model dynamics on all infected nodes
        for n in self._infected:
            events = events + self.model(n)
    
        # re-build the infected list if we need to
        if events > 0:
            self._infected = [ n for n in self._infected if self.node[n][self.DYNAMICAL_STATE] == self.INFECTED ]
        
        # if the state has changed, capture it
        if (t == 0) or (events > 0):
            ss = dict()
            for n in self.nodes_iter():
                ss[n] = self.node[n][self.DYNAMICAL_STATE]
            self._states[t] = ss
            
        return events
    
    def model( self, n ):
        '''Apply the SIR dynamics to node n. From the re-definition of _dynamics_step()
        we already know this node is infected.

        n: the node
        returns: the number of changes made'''
        events = 0
        
        # infect susceptible neighbours with probability pInfect
        for (_, m, data) in self.edges_iter(n, data = True):
            if self.node[m][self.DYNAMICAL_STATE] == self.SUSCEPTIBLE:
                if numpy.random.random() <= self._pInfect:
                    events = events + 1
                    
                    # infect the node
                    self.node[m][self.DYNAMICAL_STATE] = self.INFECTED
                    self._infected.insert(0, m)
                        
                    # label the edge we traversed as occupied
                    data[self.OCCUPIED] = True
    
        # recover with probability pRecover
        if numpy.random.random() <= self._pRecover:
            # recover the node
            events = events + 1
            self.node[n][self.DYNAMICAL_STATE] = self.RECOVERED
                
        return events
    
    def at_equilibrium( self, t ):
        '''SIR dynamics is at equilibrium if there are no more
        infected nodes left in the network or if we've exceeded
        the default simulation length.
        
        returns: True if the model has stopped'''
        if t >= 20000:
            return True
        else:
            return (len(self._infected) == 0)
            
    def dynamics( self ):
        '''Returns statistics of outbreak sizes. This skeletonises the
        network, so it can't have any further dynamics run on it.
        
        returns: a dict of statistical properties'''
        
        # run the basic dynamics
        rc = self._dynamics()
        
        # compute the limits and means
        cs = sorted(networkx.connected_components(self.skeletonise()), key = len, reverse = True)
        max_outbreak_size = len(cs[0])
        max_outbreak_proportion = (max_outbreak_size + 0.0) / self.order()
        mean_outbreak_size = numpy.mean([ len(c) for c in cs ])
        
        # add parameters and metrics for this simulation run
        rc['pInfected' ] = self._pInfected,
        rc['pRecover'] = self._pRecover,
        rc['pInfect'] = self._pInfect,
        rc['N'] = self.order(),
        rc['mean_outbreak_size'] = mean_outbreak_size,
        rc['max_outbreak_size'] = max_outbreak_size,
        rc['max_outbreak_proportion'] = max_outbreak_proportion
        rc['evolution'] = self._states
        return rc

In [11]:
# network parameters
N = 20
pEdge = 0.01
# disease parameters
pRecover = 0.002
pInfect = 0.02

# what proportion of people are initially sick
pInfected = 0.01

# how many timesteps should we run the model for
T = 1000

In [16]:
import unittest

class TestSIRResult(unittest.TestCase):
    
    def setUp(self):
        delta = 0.5
        epsilon = 0.3
        zeta = 0.4
        household_size =3
        community_size = 3
        number_of_communities = 2
        self.syn = SIRSynchronousDynamics(pInfected = pInfected, pInfect = pInfect, pRecover = 0.01,
                             g = networkx.erdos_renyi_graph(N, pEdge))
        syn_dyn = self.syn.dynamics()
        self.states = syn_dyn['evolution']
        self.changes = sorted(self.states.keys())
        
    def test_consistency(self):
        for repeat in range(0,100):
            for t in self.changes:
                susceptible_nodes = []
                infected_nodes = []
                recovered_nodes = []
                dead_nodes = []
                for node in self.states[t]:
                    if self.states[t][node]=='S':
                        susceptible_nodes.insert(0, node)
                    else:
                        if self.states[t][node]=='I':
                            infected_nodes.insert(0, node)
                        else:
                            if self.states[t][node]=='R':
                                recovered_nodes.insert(0, node)
                self.assertFalse(set(susceptible_nodes) & set(infected_nodes) & set(recovered_nodes))
            
            
    def test_unexpected_states(self):
        for repeat in range(0,100):
            once_infected = []
            once_recovered = []
            for t in self.changes:
                for node in self.states[t]:
                    if self.states[t][node]=='I':
                        once_infected.insert(0, node)
                    else:
                        if self.states[t][node]=='R':
                            once_recovered.insert(0, node)
                for n in set(once_recovered):
                    if((self.states[t][n] != 'S') and 
                                   (self.states[t][n] != 'I')):
                        self.assertTrue(True)
                    else:
                        print "R", n, once_infected,once_recovered, self.states[t][n]
                        self.assertTrue(False)

                for n in set(once_infected):
                    if(self.states[t][n] != 'S'):
                        self.assertTrue(True)
                    else:
                        print "I",n, once_infected,once_recovered, self.states[t][n]
                        self.assertTrue(False)

                                            

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSIRResult)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_consistency (__main__.TestSIRResult) ... ok
test_unexpected_states (__main__.TestSIRResult) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


In [17]:
py.test

NameError: name 'py' is not defined